### A Jupyter Notebook Demonstration for SimPy

By: The PesSIMists Group 02 (Ella Small, Kristen Stewart, Andrew Crawford, and Zack Zhang)

for: ME 396P

How to download SimPy if you don't already have it:

    Type one of the following into your terminal/console depending on if you use pip or conda

    $ python3 -m pip install simpy 
    
    or 
    
    conda install -c conda-forge simpy


In [1]:
"""The following example was adaped from:

'Simulating Real-World Processes With SimPy': https://realpython.com/simpy-simulating-with-python/#what-simulation-is

"""

# the simulation package we will use:
import simpy
#other packages for this examples:
import random
import statistics


Simulations require 3 things:
1. Establish the environment.
2. Pass in the parameters.
3. Run the simulation.

First you must establish the environment by setting a desired variable = simpy.Environment(). The enviroment moves the simulation throughout the time steps. 

Then, you’ll pass in all of the variables that will act as your parameters, which are the things you can vary to see how the system will react to changes. 

Then, it’s time to run the simulation! You can do this by calling env.run() and specifying how long you want the simulation to run for.


#### For this specific tutorial: 
Let's say we want to determine how long it will take a person to get into the next UT Football game! We can use SimPy to simulate this based off of how many people will show up, how many employees there are and average times to complete a task.

Here our environment is the stadium, and num_security are the number of people checking tickets (its a resource, meaning there are a limited number of them), and num_cashier are the number of people selling food.

In [ ]:

wait_times = []


class Stadium(object):
    def __init__(self, env, num_cashier, num_security):
        self.env = env
        self.cashier = simpy.Resource(env, num_cashier)
        self.security = simpy.Resource(env, num_security)
        
    
    def check_ticket(self, attendee):
        ''' This function tells you how long it takes to scan a ticket. env.timeout() tells simpy to trigger an event
        after a certain amount of time has passed. In this case, the event is that a ticket was scanned. 
        '''
        yield self.env.timeout(40 / 60) # in a fraction since it only takes seconds to check the ticket

    def sell_food(self, attendee):
        '''This function is if a fan wants to buy food, but we want each fan to spend a different amount of time at 
        the cashier. To do this, you use random.randint() to choose a random number between the given low and high 
        values(1 to 5 minutes in this case). Then, for each attendee, the simulation will wait for the chosen amount 
        of time.'''
        yield self.env.timeout(random.randint(1, 5))



You’ll create a function, called go_to_game(), that keeps track of when a UT fan arrives at the stadium, they’ll request a resource, wait for its process to complete, and then leave. 

There are three arguments passed to this function:

1. env: The UT fan will be controlled by the environment.
2. UT_fan: This variable tracks each person as they move through the system.
3. stadium: This parameter gives you access to the processes you defined in the overall class definition.

arrival_time will hold the time at which each fan arrives at the stadium. You can get this time using the simpy call to env.now.

You’ll want each of the processes from your stadium to have corresponding requests in go_to_game().The first process in the class is check_ticket(), which uses a security resource which needs a request to help complete the process.

In [ ]:
def go_to_game(env, UT_fan, stadium):
    # attendee arrives at the stadium
    arrival_time = env.now

    
    ''' 
    yield request: waits for a cashier to become available if all are currently in use. 
    yield env.process(): UT_fan uses an available security to complete the given process. 
    After a resource is used, it is freed up for the next agent to use (when fan enters the gates,
    the security is available to check the next ticket).
    '''
     
    with stadium.security.request() as request:
        yield request
        yield env.process(stadium.check_ticket(UT_fan))
    
    # Randomly decide if fan needs food
    if random.choice([True, False]):
        with stadium.cashier.request() as request:
            yield request
            yield env.process(stadium.sell_food(UT_fan))

    # UT fan heads into the stadium
    # Use env.now to get the time at which the attendee has finished all processes and made it to their seats.
    wait_times.append(env.now - arrival_time)



Now you’ll need to define a function to run the simulation. run_stadium() will create an instance of a stadium and generate UT fans until the simulation stops. The first thing this function should do is create an instance of a stadium, which needs the inputs of env, num_cashier, and num_security:


When you call this function, the simulation will generate 20 fans to start and begin moving them through the stadium entrance with go_to_stadium(). After that, new fans will arrive at the stadium with an interval of 6 seconds.

In [ ]:

def run_stadium(env, num_cashier, num_security):
    stadium = Stadium(env, num_cashier, num_security)

    for UT_fan in range(20):
        env.process(go_to_game(env, UT_fan, stadium))

    while True:
        #Using previous data, you learn that fans tend to arrive at the stadium every 6 seconds. Now all you have to do is tell the function to wait this long before generating a new person:
        yield env.timeout(0.10)  # Wait a bit before generating a new person

        #After waiting, the function should increment UT_fan by 1 and generate the next person. The generator function is the same one you used to initialize the first 20 fans
        UT_fan += 1
        env.process(go_to_game(env, UT_fan, stadium))



The list wait_times will have the total amount of time it took each UT fan to make it to their seat. Now you’ll want to calculate the average time a fan spends from the time they arrive to the time they finish checking their ticket.

In [ ]:

def get_average_wait_time(wait_times):
    average_wait = statistics.mean(wait_times)
    # Pretty print the results
    minutes, frac_minutes = divmod(average_wait, 1)
    seconds = frac_minutes * 60
    return round(minutes), round(seconds)



These variables are the parameters that you can change to see how the simulation changes. If a top-10 matchup has customers lining up around the block, we want to ensure an appropriate number of security are working and prevent a long wait at concessions by making sure the number of cashiers is sufficient.

Using simpy, you are able to change the values of these parameters to try out different scenarios using a helper function to get user input:

In [ ]:

def get_user_input():
    num_cashier = input("Input number of cashiers working: ")
    num_security = input("Input number of security working: ")
    params = [num_cashier, num_security]
    if all(str(i).isdigit() for i in params):  # Check input is valid
        params = [int(x) for x in params]
    else:
        print(
            "Could not parse input. Simulation will use default values:",
            "\n 1 cashier, 1 security.",
        )
        params = [1, 1]
    return params




Here’s how main() works:

1. Get user input.
2. Create environment and save it as env, which will move the simulation through each time step.
3. Tell simpy to run run_stadium(), which creates the stadium environment and generates UT fans to move.
4. Determine how long you want the simulation to run. As a default value, the simulation is set to run for 90 minutes.
5. Store the output of get_average_wait_time() in two variables, mins and secs.
6. Use print() to show the results to the user.

In [ ]:
def main():
    # Setup
    #random.seed(42)
    num_cashier, num_security = get_user_input()

    # Run the simulation
    env = simpy.Environment()
    env.process(run_stadium(env, num_cashier, num_security))
    env.run(until=90)

    # View the results
    mins, secs = get_average_wait_time(wait_times)
    print(
        "Running simulation...",
        f"\nThe average wait time is {mins} minutes and {secs} seconds.",
    )


if __name__ == "__main__":
    main()
